In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from utils_flat_files import FlatFiles

# #S3 paths
#INPUT_PATH = r's3://mercy-locust-covid19-in-dev/inbound/sourcedata/Spatial/'
#OUTPUT_PATH = r's3://mercy-locust-covid19-out-dev/location_dim/'
#local paths
INPUT_PATH = r'data/input/'
OUTPUT_PATH = r'data/output/'

In [2]:
file_name_1='UA_Eggs_AF.xlsx'
file_name_2='UA_Milk_AF.xlsx'
file_name_3='UA_Mutt_AF.xlsx'
file_name_4='UA_Pork_AF.xlsx'
file_name_5='UA_Pou_AF.xlsx'
file_name_6='UrbanAreas_Beef_Africa.xlsx'

In [3]:
df_egg = pd.read_excel(INPUT_PATH + file_name_1)
df_milk = pd.read_excel(INPUT_PATH + file_name_2)
df_mutt = pd.read_excel(INPUT_PATH + file_name_3)
df_pork = pd.read_excel(INPUT_PATH + file_name_4)
df_pou = pd.read_excel(INPUT_PATH + file_name_5)
df_beef = pd.read_excel(INPUT_PATH + file_name_6)

In [4]:
df_egg=df_egg.drop(['NEW_URBID','ISO3','Continent','SQKM_FINAL'], axis=1)
df_milk=df_milk.drop(['NEW_URBID','ISO3','Continent','SQKM_FINAL','WB_Region_2010'], axis=1)
df_mutt=df_mutt.drop(['NEW_URBID','ISO3','Continent','SQKM_FINAL','WB_Region_2010'], axis=1)
df_pork=df_pork.drop(['NEW_URBID','ISO3','Continent','SQKM_FINAL','WB_Region_2010'], axis=1)
df_pou=df_pou.drop(['NEW_URBID','ISO3','Continent','SQKM_FINAL','WB_Region_2010'], axis=1)
df_beef=df_beef.drop(['NEW_URBID','ISO3','Continent','SQKM_FINAL','WB_Region_2010'], axis=1)

In [5]:
countries=['Ethiopia','Kenya','Somalia','Uganda']

In [6]:
df_egg=df_egg[df_egg['ADM0_Name'].isin(countries)]
df_milk=df_milk[df_milk['ADM0_Name'].isin(countries)]
df_mutt=df_mutt[df_mutt['ADM0_Name'].isin(countries)]
df_pork=df_pork[df_pork['ADM0_Name'].isin(countries)]
df_pou=df_pou[df_pou['ADM0_Name'].isin(countries)]
df_beef=df_beef[df_beef['ADM0_Name'].isin(countries)]

In [7]:
df_egg = df_egg.rename(columns={'Egg_Cons00': 'Cons00', 'Egg_Cons30': 'Cons30'})
df_milk = df_milk.rename(columns={'Milk_Cons00': 'Cons00', 'Milk_Cons30': 'Cons30'})
df_mutt = df_mutt.rename(columns={'Mut_Cons00': 'Cons00', 'Mut_Cons30': 'Cons30'})
df_pork = df_pork.rename(columns={'Pork_Cons00': 'Cons00', 'Pork_Cons30': 'Cons30'})
df_pou = df_pou.rename(columns={'Pou_Cons00': 'Cons00', 'Pou_Cons30': 'Cons30'})
df_beef = df_beef.rename(columns={'Beef_Cons00': 'Cons00', 'Beef_Cons30': 'Cons30'})


In [8]:
df_egg.insert(7, 'dm_commodity_name','egg consumption') #insert column at specified location (column 7 here)
df_milk.insert(7, 'dm_commodity_name','milk consumption')
df_mutt.insert(7, 'dm_commodity_name','mutton consumption')
df_pork.insert(7, 'dm_commodity_name','pork consumption')
df_pou.insert(7, 'dm_commodity_name','poultry consumption')
df_beef.insert(7, 'dm_commodity_name','beef consumption')

In [9]:
frames= pd.concat([df_egg, df_milk, df_mutt, df_pork, df_pou, df_beef]) #called 'frames' for all dataframes
frames.head()

,NAME,ADM0_Name,ES00POP,UA_pop30,Cons00,Cons30,Egg_UA_T,dm_commodity_name,Mlk_UA_T,Mut_UA_T,Pork_UA_T,Pou_UA_T,Beef_UA_T
1207,ADIGRAT,Ethiopia,48231,171696.768539,30.8042,234.5760,0.234490,egg consumption,NaN,NaN,NaN,NaN,NaN
1208,ADWA,Ethiopia,31605,112510.136005,20.1806,153.6770,0.153621,egg consumption,NaN,NaN,NaN,NaN,NaN
1209,AXUM,Ethiopia,34994,124574.583116,22.3766,170.4000,0.170338,egg consumption,NaN,NaN,NaN,NaN,NaN
1210,ENDASILASIE,Ethiopia,32572,115952.543900,13.6588,48.3752,0.039950,egg consumption,NaN,NaN,NaN,NaN,NaN
1211,MAKALLE,Ethiopia,132545,471844.833945,130.7020,995.3040,0.994939,egg consumption,NaN,NaN,NaN,NaN,NaN


In [10]:
conditions = [
    (frames['dm_commodity_name'] == 'egg consumption'), #if entry satisfied multiple conditions the first one satisfied is taken
    (frames['dm_commodity_name'] == 'milk consumption'),
    (frames['dm_commodity_name'] == 'mutton consumption'),
    (frames['dm_commodity_name'] == 'pork consumption'),
   (frames['dm_commodity_name'] == 'poultry consumption'),
    (frames['dm_commodity_name'] == 'beef consumption')]
    
choices = ['21', '20', '24', '25', '23', '22'] 
frames['measureID'] = np.select(conditions, choices) #assign an ID to each of the different types of conditions

In [11]:
#Now we need to map locations within countries to their location ID.

location=pd.read_csv(INPUT_PATH + 'cities_mapping_all_countries.csv', sep='|') #------------------not sure what csv this is???
print(len(location['City']))
frames['NAME']= frames['NAME'].str.lower()
frames["NAME"]= frames["NAME"].str.strip()
location['City']= location['City'].str.lower()
print(len(location['City']))
location["City"]= location["City"].str.strip()

locID = []
count = 0
for line in frames['NAME']: #match names of cities to location IDs
    mask = np.isin(location['City'],line)
    if sum(mask):
            flag = 0
            for i in range(len(location['City'])):
                if flag == 1:
                    #nothing
                    0
                elif location['City'][i] == line: #TODO : map all locations, currently some are removed that do not have a locationID
                    locID.append(location['locationID'][i])
                    flag = 1
    else:
        locID.append(np.nan)
    count += 1

#add locID to frames
frames['locationID'] = locID
#check
frames.head()

528
528


,NAME,ADM0_Name,ES00POP,UA_pop30,Cons00,Cons30,Egg_UA_T,dm_commodity_name,Mlk_UA_T,Mut_UA_T,Pork_UA_T,Pou_UA_T,Beef_UA_T,measureID,locationID
1207,adigrat,Ethiopia,48231,171696.768539,30.8042,234.5760,0.234490,egg consumption,NaN,NaN,NaN,NaN,NaN,21,NaN
1208,adwa,Ethiopia,31605,112510.136005,20.1806,153.6770,0.153621,egg consumption,NaN,NaN,NaN,NaN,NaN,21,NaN
1209,axum,Ethiopia,34994,124574.583116,22.3766,170.4000,0.170338,egg consumption,NaN,NaN,NaN,NaN,NaN,21,ETH.11_1
1210,endasilasie,Ethiopia,32572,115952.543900,13.6588,48.3752,0.039950,egg consumption,NaN,NaN,NaN,NaN,NaN,21,ETH.11_1
1211,makalle,Ethiopia,132545,471844.833945,130.7020,995.3040,0.994939,egg consumption,NaN,NaN,NaN,NaN,NaN,21,ETH.11_1


In [12]:
#==========================================New way of predicting consumption==========================================
#Functions to detect which districts are in the same regions and sum those populations

import re
def split_locationID(locationID):
    split = re.split('[_ .]',locationID) #should have 4 strings
    country = split[0]
    area = split[1]
    district = split[2]
    region = split[3]
    return country, area, district, region

In [13]:
#Function to sum population of all districts within a region
def sum_regions(filename):
    pop = pd.read_csv(INPUT_PATH + filename, sep = '|')
    pop = pop.dropna()    
    
    splits = np.empty((len(pop['locationID']),4), dtype = object)
    regions = []    
    pop_sums = []
    i = 0
    temp = []
    for line in pop['locationID']:
        splits[i] = split_locationID(line)
        regions.append(splits[i][0] + '.' + splits[i][1] + '_' + splits[i][3])
        if i > 0:
            if splits[i][0] == splits[i-1][0] and splits[i][1] == splits[i-1][1] and splits[i][3] ==splits[i-1][3]:
                temp.append(pop['value'][i]) #keep track of all districts in a region
            else:
                if temp != []:
                    pop_sums.append(sum(np.float64(temp)))
                    temp = [] #set back to empty
                else:
                    pop_sums.append(pop['value'][i]) #if the only district in its region
                    temp = [] #set back to empty
        i += 1
    #Get last line:
    lastline = pop['locationID'][len(pop['locationID'])]
    split = split_locationID(lastline)
    regions.append(split[0] + '.' + split[1] + '_' + split[3])
    pop_sums.append(pop['value'][i]) 
    
    return pop_sums, regions

In [14]:
#Create prediction in consumption based on current population, not only on two data points of consumption.

pop00, regions = sum_regions("population_table_2000.csv")
pop14, regions14 = sum_regions("population_table_2014.csv")
pop16, regions16 = sum_regions("population_table_2016.csv")
pop17, regions17 = sum_regions("population_table_2017.csv")
pop18, regions18 = sum_regions("population_table_2018.csv")
pop20, regions20 = sum_regions("population_table_2020.csv")

print(len(pop00),len(pop14),len(pop16),len(pop17),len(pop18),len(pop20)) #they should have the same number of regions each

134 134 134 134 134 134


In [15]:
#Now we need to fit a curve to this data, for each region
from scipy.optimize import curve_fit

def func(x, r, a):
    return a * ((1+ r)**x) #same formula used above

time = [0,14,16,17,18,20]

curve_params = []
for region in range(len(pop00)):
    series = [ pop00[region], pop14[region], pop16[region], pop17[region], pop18[region], pop20[region] ]
    params, cov = curve_fit(func,time,series,maxfev = 1000) 
    curve_params.append(params[0])


In [23]:
#Work out cons20 for all regions:

#Get region tags
print(regions == regions14 == regions16 == regions17 == regions18 == regions20) #yes they are all the same
#Remove duplicates from region tags:
all_regions = []
[all_regions.append(x) for x in regions if x not in all_regions]
print("done")

#Work out cons20
demand = frames[pd.notnull(frames["locationID"])] #remove any columns that are nan (do this after adding IDs)

region_params = []
for row in demand['locationID']:    #use demand not frames as we have already removed nan from demand
    index = all_regions.index(row)
    region_params.append(curve_params[index] ) #save the curve for that region

demand['region_params'] = region_params #so that their indices match when plotting
    
cons14,cons15,cons16,cons17,cons18,cons19,cons20 = [], [], [], [], [], [], []

i=0
for row in demand['Cons00']:
    cons14.append( func(14, region_params[i], row) ) 
    cons15.append( func(15, region_params[i], row) ) 
    cons16.append( func(16, region_params[i], row) ) 
    cons17.append( func(17, region_params[i], row) ) 
    cons18.append( func(18, region_params[i], row) ) 
    cons19.append( func(19, region_params[i], row) ) 
    cons20.append( func(20, region_params[i], row) ) 

    i += 1

#Create date IDs for each of the different time predictions:

demand2 = demand[['measureID','locationID','Cons00','dm_commodity_name']]

df00 = demand2
df00 = df00.rename(columns={'Cons00': 'value'})

demand2 = demand2.drop(['Cons00'], axis=1)

df14 = pd.concat([ pd.DataFrame(cons14, columns = ['value'])  , demand2.reset_index(drop=True) ], axis = 1)
df15 = pd.concat([ pd.DataFrame(cons15, columns = ['value'])  , demand2.reset_index(drop=True) ], axis = 1)
df16 = pd.concat([ pd.DataFrame(cons16, columns = ['value'])  , demand2.reset_index(drop=True) ], axis = 1)
df17 = pd.concat([ pd.DataFrame(cons17, columns = ['value'])  , demand2.reset_index(drop=True) ], axis = 1)
df18 = pd.concat([ pd.DataFrame(cons18, columns = ['value'])  , demand2.reset_index(drop=True) ], axis = 1)
df19 = pd.concat([ pd.DataFrame(cons19, columns = ['value'])  , demand2.reset_index(drop=True) ], axis = 1)
df20 = pd.concat([ pd.DataFrame(cons20, columns = ['value'])  , demand2.reset_index(drop=True) ], axis = 1)

#Concat each of them with demand so they have location ID and measureID and commodity name
date00 = [2000 for i in range(len(df00['value']))]
df00.insert(1,'date',date00)
df00 = FlatFiles().add_date_id(df00,'date')

date14 = [2014 for i in range(len(cons14))]
df14.insert(1,'date',date14)
df14 = FlatFiles().add_date_id(df14,'date')

date15 = [2015 for i in range(len(cons15))]
df15.insert(1,'date',date15)
df15 = FlatFiles().add_date_id(df15,'date')

date16 = [2016 for i in range(len(cons16))]
df16.insert(1,'date',date16)
df16 = FlatFiles().add_date_id(df16,'date')

date17 = [2017 for i in range(len(cons17))]
df17.insert(1,'date',date17)
df17 = FlatFiles().add_date_id(df17,'date')

date18 = [2018 for i in range(len(cons18))]
df18.insert(1,'date',date18)
df18 = FlatFiles().add_date_id(df18,'date')

date19 = [2019 for i in range(len(cons19))]
df19.insert(1,'date',date19)
df19 = FlatFiles().add_date_id(df19,'date')

date20 = [2020 for i in range(len(cons20))]
df20.insert(1,'date',date20)
df20 = FlatFiles().add_date_id(df20,'date')

demand_final= pd.concat([df00, df14, df15, df16, df17, df18, df19, df20]) #called 'frames' for all dataframes
demand_final =demand_final.drop(['date'], axis=1)

demand_final = demand_final[['measureID','dateID','locationID','value','dm_commodity_name']]


demand_final.insert(0, 'factID', [str('DEF_'+str(i+1)) for i in range(len(demand_final['value']))] ) #insert at first columns

True
done


<ipython-input-23-ce7d845eba48>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demand['region_params'] = region_params #so that their indices match when plotting


In [24]:
demand_final[3000:3100]

,factID,measureID,dateID,locationID,value,dm_commodity_name
684,DEF_3001,20,20140101,ETH.10_1,33.989224,milk consumption
685,DEF_3002,20,20140101,ETH.10_1,35.541482,milk consumption
686,DEF_3003,20,20140101,ETH.8_1,333.721821,milk consumption
687,DEF_3004,20,20140101,ETH.8_1,723.531654,milk consumption
688,DEF_3005,20,20140101,ETH.10_1,522.679213,milk consumption
...,...,...,...,...,...,...
779,DEF_3096,24,20140101,ETH.3_1,150.827725,mutton consumption
780,DEF_3097,24,20140101,ETH.3_1,1.030054,mutton consumption
781,DEF_3098,24,20140101,ETH.3_1,6.236184,mutton consumption
782,DEF_3099,24,20140101,ETH.3_1,130.774026,mutton consumption


In [25]:
#Create final output:

#Use utils function to add today's date to the filename
out = FlatFiles().export_output_w_date(demand_final, "Demand")

ArrowException: Unsupported compression: UNCOMPRESSED

Exception ignored in: 'pyarrow._parquet.ParquetWriter._set_compression_props'
Traceback (most recent call last):
  File "pyarrow\_parquet.pyx", line 973, in pyarrow._parquet.check_compression_name
pyarrow.lib.ArrowException: Unsupported compression: UNCOMPRESSED


Dataframe exported to parquet format
Dataframe exported to csv format


In [19]:
def graph(number):
    time_og = [2000,2015,2016,2017,2018,2019,2020,2030]
    time_pop = [2000,2015,2016,2017,2018,2019,2020]
    cont_time_plot = [x for x in range(2000,2030)]
    cont_time = [x for x in range(30)]

    original = [demand['Cons00'][number], demand['value15'][number],demand['value16'][number],demand['value17'][number],demand['value18'][number],demand['value19'][number], demand['value'][number], demand['Cons30'][number] ]
    population = [demand['Cons00'][number], demand['Cons15'][number], demand['Cons16'][number], demand['Cons17'][number], demand['Cons18'][number], demand['Cons19'][number], demand['Cons20'][number] ]
    plt.plot(time_og, original,"ob", label= None) #"original method (data points)")
    plt.plot(cont_time_plot, [get_exp_growth(t,number = number) for t in cont_time], label="original method") 

    plt.plot(time_pop, population,"ob", label= None)  #"population based method (data points)")
    plt.plot(cont_time_plot, func(cont_time, demand['region_params'][number], demand['Cons00'][number]), label="population based method") 

    plt.xlabel("Years")
    plt.ylabel("Consumption (Tonnes)")
    plt.title("Consumption of " + str(demand['locationID'][number]) + " over Time")
    plt.legend()
    plt.show()


In [20]:
graph(2)

KeyError: 2

In [ ]:
graph(3)

In [ ]:
graph(30)

In [ ]:
graph(130)

In [ ]:
graph(1035)

In [ ]:
graph(2103)

In [ ]:
graph(2312)